<a href="https://colab.research.google.com/github/xicodeivid/lol_analyst/blob/main/Request_Riot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Estrutura simples
Ivocador
Campeões
Partidas
Participantes
Bans
Resumo da partida por time
Resumo da partida por participante

In [ ]:
import requests as r
import pandas as pd
import json
from datetime import datetime

In [ ]:
key = 'RGAPI-84dc7fde-9e72-4b84-82e0-ced9a4b322fd'
evoc = 'VermelhodeSangue'

In [ ]:
def info_evoc(evoc,key):
  link_evoc = 'https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + evoc + '?api_key=' + key
  return r.get(link_evoc).json()

In [ ]:
def partidas(accountid,key):
  link_part = 'https://br1.api.riotgames.com/lol/match/v4/matchlists/by-account/' + accountid + '?api_key=' + key
  part = pd.DataFrame(r.get(link_part).json()['matches'])
  part['accountid'] = accountid
  #part['timestamp'] = datetime.fromtimestamp(part['timestamp'] * 0.001)
  return part

In [ ]:
def partida_det(gameid,key):
  link_partd = 'https://br1.api.riotgames.com/lol/match/v4/matches/' + gameid + '?api_key=' + key
  xy = pd.io.json.json_normalize(r.get(link_partd).json())
  part_b = xy[['gameId','platformId','gameCreation','gameDuration','queueId','mapId','seasonId','gameVersion','gameMode','gameType']]
  part_t = xy[['gameId','teams']]
  part_p = xy[['gameId','participants']]
  part_pi = xy[['gameId','participantIdentities']]
  return [part_b, part_t, part_p, part_pi]

In [ ]:
def captura_bans(lista):
  bans = lista['bans']
  bans100 = pd.DataFrame(bans[0])
  bans100['teamId'] = 100
  bans100 = bans100[['teamId','championId','pickTurn']]
  bans200 = pd.DataFrame(bans[1])
  bans200['teamId'] = 200
  bans200 = bans200[['teamId','championId','pickTurn']]
  bans = pd.concat([bans100, bans200], ignore_index=True)
  return bans

In [ ]:
i_evoc = info_evoc(evoc,key) #informações basicas do evocador
i_part = partidas(i_evoc['accountId'],key) #historico de partidas
i_partd = partida_det(str(i_part['gameId'][0]),key) #detalhes da partida

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
pd.DataFrame(json.loads(i_partd[1]['teams'].to_json())["0"])

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans
0,100,Fail,True,True,False,False,True,True,7,0,0,4,0,2,0,"[{'championId': 11, 'pickTurn': 1}, {'champion..."
1,200,Win,False,False,True,True,False,False,8,1,1,2,0,0,0,"[{'championId': 11, 'pickTurn': 6}, {'champion..."


In [ ]:
part_det_b = pd.DataFrame(json.loads(i_partd[1]['teams'].to_json())["0"])
part_det_b['gameId'] = i_partd[0]["gameId"][0]
bans = part_det_b['bans']
part_det = part_det_b[['gameId','teamId','win','firstBlood','firstTower','firstInhibitor','firstBaron','firstDragon','firstRiftHerald','towerKills','inhibitorKills','baronKills','dragonKills','vilemawKills','riftHeraldKills','dominionVictoryScore']]

In [ ]:
bans = captura_bans(part_det_b)
bans

,teamId,championId,pickTurn
0,100,11,1
1,100,157,2
2,100,-1,3
3,100,32,4
4,100,67,5
5,200,11,6
6,200,81,7
7,200,120,8
8,200,777,9
9,200,105,10


In [ ]:
part_det

,gameId,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore
0,2243379942,100,Fail,True,True,False,False,True,True,7,0,0,4,0,2,0
1,2243379942,200,Win,False,False,True,True,False,False,8,1,1,2,0,0,0


In [ ]:
part_1 = pd.DataFrame(json.loads(i_partd[2]['participants'].to_json())["0"])

In [ ]:
resumo = pd.DataFrame(json.loads(part_1['stats'].to_json())).reset_index(level=0)
resumo = resumo.rename(columns={'index': 'evento'})
resumo['gameId'] = i_partd[0]["gameId"][0]
resumo = resumo[['gameId','evento','0','1','2','3','4','5','6','7','8','9']]
resumo

,gameId,evento,0,1,2,3,4,5,6,7,8,9
0,2243379942,participantId,1,2,3,4,5,6,7,8,9,10
1,2243379942,win,False,False,False,False,False,True,True,True,True,True
2,2243379942,item0,3153,6692,1055,6664,3860,6672,3135,6692,3853,6691
3,2243379942,item1,3124,3153,3085,1054,6653,3036,6656,2031,3157,1006
4,2243379942,item2,3006,3006,6672,3009,3158,3006,3157,3047,3020,3158
...,...,...,...,...,...,...,...,...,...,...,...,...
99,2243379942,perkPrimaryStyle,8000,8100,8100,8400,8200,8000,8100,8000,8200,8000
100,2243379942,perkSubStyle,8100,8000,8000,8000,8300,8100,8000,8100,8100,8100
101,2243379942,statPerk0,5005,5005,5005,5005,5007,5005,5007,5008,5008,5008
102,2243379942,statPerk1,5008,5008,5008,5008,5008,5008,5002,5008,5008,5008


In [ ]:
part_1.loc[part_1.participantId == 1]['stats'][0]

{'assists': 6,
 'champLevel': 17,
 'combatPlayerScore': 0,
 'damageDealtToObjectives': 6347,
 'damageDealtToTurrets': 2949,
 'damageSelfMitigated': 14617,
 'deaths': 7,
 'doubleKills': 0,
 'firstBloodAssist': False,
 'firstBloodKill': True,
 'firstInhibitorAssist': False,
 'firstInhibitorKill': False,
 'firstTowerAssist': False,
 'firstTowerKill': False,
 'goldEarned': 15182,
 'goldSpent': 14300,
 'inhibitorKills': 0,
 'item0': 3153,
 'item1': 3124,
 'item2': 3006,
 'item3': 3085,
 'item4': 6672,
 'item5': 1036,
 'item6': 3363,
 'killingSprees': 2,
 'kills': 9,
 'largestCriticalStrike': 367,
 'largestKillingSpree': 4,
 'largestMultiKill': 1,
 'longestTimeSpentLiving': 858,
 'magicDamageDealt': 34124,
 'magicDamageDealtToChampions': 11000,
 'magicalDamageTaken': 17075,
 'neutralMinionsKilled': 12,
 'neutralMinionsKilledEnemyJungle': 4,
 'neutralMinionsKilledTeamJungle': 8,
 'objectivePlayerScore': 0,
 'participantId': 1,
 'pentaKills': 0,
 'perk0': 8005,
 'perk0Var1': 1754,
 'perk0Var2'

In [ ]:
pd.DataFrame(json.loads(i_partd[1]['teams'].to_json())["0"])

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans
0,100,Fail,False,False,False,False,False,False,2,0,0,0,0,0,0,"[{'championId': 11, 'pickTurn': 1}, {'champion..."
1,200,Win,True,True,False,False,True,False,3,0,0,2,0,0,0,"[{'championId': 234, 'pickTurn': 6}, {'champio..."


In [ ]:
x.keys()

NameError: ignored

In [ ]:
pd.DataFrame(x['participants'])

NameError: ignored

In [ ]:
df = pd.io.json.json_normalize(x['participants'], 'stats', ['participantId','teamId','championId','spell1Id','spell2Id'],record_prefix='stats_')

NameError: ignored

In [ ]:
df

NameError: ignored

In [ ]:
pd.pivot_table(df, values='stats_0', columns=['participantId','teamId','championId','spell1Id','spell2Id'], aggfunc=lambda x: len(x.unique()))
#pd.pivot_table(df, values='stats_0', index=['A', 'B'], columns=['C'], aggfunc=np.sum)

AttributeError: ignored

In [ ]:
x['teams']

NameError: ignored

In [ ]:
link_camp = 'http://ddragon.leagueoflegends.com/cdn/10.1.1/data/pt_BR/champion.json'
r3 = r.get(link_camp).json()

In [ ]:
link_camp = 'http://ddragon.leagueoflegends.com/cdn/10.1.1/data/pt_BR/champion.json'
r3 = r.get(link_camp).json()
campeoes = [x for x in r3['data'].keys()]
campeoes_pd = pd.DataFrame()
n = 0

for x in campeoes:
  nome = {'nome' : x}
  id_camp = {'id' : r3['data'][x]['key']}
  info_camp = r3['data'][x]['info']
  tags_camp = {'tags_camp' : ', '.join(r3['data'][x]['tags'])}
  stats_camp = r3['data'][x]['stats']
  version_camp = {'version_camp' : r3['data'][x]['version']}

  dic = {}
  for d in (nome,id_camp,info_camp,tags_camp,stats_camp,version_camp):
    dic.update(d)
  
  testepd = pd.DataFrame(dic, index=[n])
  campeoes_pd = campeoes_pd.append(testepd)
  n = n + 1

In [ ]:
campeoes_pd.sort_values(by=['hpperlevel','hp'], ascending=False)

,nome,id,attack,defense,magic,difficulty,tags_camp,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,version_camp
112,Sylas,517,3,4,8,5,"Mage, Assassin",525.00,115,280.0,50.0,340,27.0,4.0,32.0,1.75,175,9.0,0.90,7.0,0.80,0,0,61.0,3.00,3.500,0.645,10.1.1
31,Galio,3,1,10,6,5,"Tank, Mage",562.00,112,500.0,40.0,335,24.0,3.5,32.0,1.25,150,8.0,0.80,9.5,0.70,0,0,59.0,3.50,1.500,0.625,10.1.1
3,Alistar,12,6,9,5,7,"Tank, Support",575.00,106,350.0,40.0,330,44.0,3.5,32.1,1.25,125,8.5,0.85,8.5,0.80,0,0,62.0,3.75,2.125,0.625,10.1.1
114,TahmKench,223,3,9,6,5,"Support, Tank",600.00,100,325.0,40.0,335,47.0,3.5,32.1,1.25,175,6.5,0.55,8.0,1.00,0,0,56.0,3.20,2.500,0.658,10.1.1
20,Darius,122,9,5,1,2,"Fighter, Tank",582.24,100,263.0,37.5,340,39.0,4.0,32.1,1.25,175,10.0,0.95,6.6,0.35,0,0,64.0,5.00,1.000,0.625,10.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Sion,14,5,9,3,5,"Tank, Fighter",545.00,73,330.0,42.0,345,32.0,3.0,32.1,1.25,175,7.5,0.80,8.0,0.60,0,0,68.0,4.00,1.300,0.679,10.1.1
42,Janna,40,3,5,7,7,"Support, Mage",500.00,70,350.0,64.0,315,28.0,3.8,30.0,0.50,550,5.5,0.55,11.5,0.40,0,0,46.0,1.50,2.950,0.625,10.1.1
141,Yuumi,350,5,1,8,2,"Support, Mage",480.00,70,400.0,45.0,330,25.0,3.0,25.0,0.30,500,7.0,0.55,10.0,0.40,0,0,55.0,3.10,1.000,0.625,10.1.1
59,Kled,240,8,2,2,7,"Fighter, Tank",340.00,70,100.0,0.0,345,35.0,4.0,32.1,1.25,125,6.0,0.75,0.0,0.00,0,0,65.0,3.50,3.500,0.625,10.1.1
